In [1]:
import heapq

class MazeSolver:
    def __init__(self, maze):
        self.original_maze = [row[:] for row in maze]  # Deep copy to preserve original
        self.maze = [row[:] for row in maze]
        self.start, self.goal = self.find_positions()
        self.rows = len(maze)
        self.cols = len(maze[0])
        self.process_maze()

    def find_positions(self):
        start = goal = None
        for i in range(len(self.maze)):
            for j in range(len(self.maze[0])):
                if self.maze[i][j] == 'A':
                    start = (i, j)
                elif self.maze[i][j] == 'B':
                    goal = (i, j)
        return start, goal

    def manhattan(self, a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def process_maze(self):
        for i in range(self.rows):
            for j in range(self.cols):
                if self.maze[i][j] in ['A', 'B']:
                    self.maze[i][j] = 0

    def get_neighbors(self, pos):
        neighbors = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for dx, dy in directions:
            nx, ny = pos[0] + dx, pos[1] + dy
            if 0 <= nx < self.rows and 0 <= ny < self.cols and self.maze[nx][ny] != 1:
                neighbors.append((nx, ny))
        return neighbors

    def greedy_bfs(self):
        visited = set()
        came_from = {}

        heap = []
        heapq.heappush(heap, (self.manhattan(self.start, self.goal), self.start))

        while heap:
            _, current = heapq.heappop(heap)

            if current == self.goal:
                break

            if current in visited:
                continue
            visited.add(current)

            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    heapq.heappush(heap, (self.manhattan(neighbor, self.goal), neighbor))
                    if neighbor not in came_from:
                        came_from[neighbor] = current

        # Reconstruct path
        path = []
        curr = self.goal
        while curr != self.start:
            path.append(curr)
            curr = came_from.get(curr)
            if curr is None:
                print("No path found.")
                return []
        path.append(self.start)
        path.reverse()
        return path

    def visualize_path(self, path):
        maze_copy = [row[:] for row in self.original_maze]
        for r, c in path:
            if (r, c) != self.start and (r, c) != self.goal:
                maze_copy[r][c] = '*'

        for row in maze_copy:
            print(' '.join(str(cell) for cell in row))
        print()

    def visualize_manhattan(self):
        matrix = []
        for i in range(self.rows):
            row = []
            for j in range(self.cols):
                if self.original_maze[i][j] == 1:
                    row.append("█")
                elif (i, j) == self.start:
                    row.append('A')
                elif (i, j) == self.goal:
                    row.append('B')
                else:
                    row.append(self.manhattan((i, j), self.goal))
            matrix.append(row)
        return matrix

    @staticmethod
    def print_matrix(matrix):
        for row in matrix:
            row_str = ""
            for val in row:
                if val == "█":
                    row_str += " █ "
                elif val == 'A' or val == 'B':
                    row_str += f" {val} "
                else:
                    row_str += f"{val:2d} "
            print(row_str)
        print()


In [13]:
# Define the new maze
maze5 = [
    ['A', 0,  1,  0,  0,  0, 1, 0, 0, 1, 0, 0],
    [0,   0,  1,  0,  1,  0, 1, 0, 1, 0, 0, 0],
    [1,   0,  0,  0,  1,  0, 0, 0, 1, 1, 1, 0],
    [0,   1,  1,  0,  0,  1, 1, 0, 0, 0, 1, 0],
    [0,   0,  0,  0,  1,  0, 1, 1, 1, 0, 0, 0],
    [1,   1,  1,  0,  1,  0, 0, 0, 1, 1, 1, 0],
    [0,   0,  1,  0,  0,  0, 1, 0, 0, 0, 1, 0],
    [0,   1,  0,  1,  1,  0, 1, 1, 1, 0, 1, 0],
    [0,   0,  0,  0,  0,  0, 0, 0, 1, 0, 0, 0],
    [1,   1,  1,  1,  1,  1, 0, 1, 0, 1, 1, 0],
    [0,   0,  0,  0,  0,  0, 0, 1, 0, 0, 0, 0],
    [0,   1,  1,  1,  1,  1, 1, 1, 1, 1, 1,'B'],
]

# Initialize solver
solver = MazeSolver(maze5)

# Run greedy best-first search
path = solver.greedy_bfs()

# Output
print("Path from A to B:")
print(path)
solver.visualize_path(path)

# Optional: visualize manhattan distances
print("Manhattan Distance Matrix:")
matrix = solver.visualize_manhattan()
solver.print_matrix(matrix)


Path from A to B:
[(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3), (6, 4), (6, 5), (5, 5), (5, 6), (5, 7), (6, 7), (6, 8), (6, 9), (7, 9), (8, 9), (8, 10), (8, 11), (9, 11), (10, 11), (11, 11)]
A * 1 0 0 0 1 0 0 1 0 0
0 * 1 0 1 0 1 0 1 0 0 0
1 * * * 1 0 0 0 1 1 1 0
0 1 1 * 0 1 1 0 0 0 1 0
0 0 0 * 1 0 1 1 1 0 0 0
1 1 1 * 1 * * * 1 1 1 0
0 0 1 * * * 1 * * * 1 0
0 1 0 1 1 0 1 1 1 * 1 0
0 0 0 0 0 0 0 0 1 * * *
1 1 1 1 1 1 0 1 0 1 1 *
0 0 0 0 0 0 0 1 0 0 0 *
0 1 1 1 1 1 1 1 1 1 1 B

Manhattan Distance Matrix:
 A 21  █ 19 18 17  █ 15 14  █ 12 11 
21 20  █ 18  █ 16  █ 14  █ 12 11 10 
 █ 19 18 17  █ 15 14 13  █  █  █  9 
19  █  █ 16 15  █  █ 12 11 10  █  8 
18 17 16 15  █ 13  █  █  █  9  8  7 
 █  █  █ 14  █ 12 11 10  █  █  █  6 
16 15  █ 13 12 11  █  9  8  7  █  5 
15  █ 13  █  █ 10  █  █  █  6  █  4 
14 13 12 11 10  9  8  7  █  5  4  3 
 █  █  █  █  █  █  7  █  5  █  █  2 
12 11 10  9  8  7  6  █  4  3  2  1 
11  █  █  █  █  █  █  █  █  █  █  B 



1.5 time in manhattan


In [3]:
from heapq import heappush, heappop

class MazeSolver:
    def __init__(self, maze):
        self.maze = maze
        self.rows = len(maze)
        self.cols = len(maze[0])
        self.start = self.find_symbol('A')
        self.goal = self.find_symbol('B')

    # Locate start/goal
    def find_symbol(self, symbol):
        for r in range(self.rows):
            for c in range(self.cols):
                if self.maze[r][c] == symbol:
                    return (r, c)
        return None

    # Greedy Best-First Search
    def greedy_bfs(self):
        start, goal = self.start, self.goal
        frontier = []
        heappush(frontier, (self.manhattan(start, goal), start))
        came_from = {start: None}
        visited = set()

        while frontier:
            _, current = heappop(frontier)
            if current == goal:
                break
            if current in visited:
                continue
            visited.add(current)

            for dr, dc in [(1,0), (-1,0), (0,1), (0,-1)]:
                nr, nc = current[0] + dr, current[1] + dc
                if self.is_valid(nr, nc) and (nr, nc) not in visited:
                    heappush(frontier, (self.manhattan((nr, nc), goal), (nr, nc)))
                    if (nr, nc) not in came_from:
                        came_from[(nr, nc)] = current

        # Reconstruct path
        path = []
        node = goal
        while node:
            path.append(node)
            node = came_from.get(node)
        return path[::-1]

    def is_valid(self, r, c):
        return (0 <= r < self.rows and 0 <= c < self.cols
                and self.maze[r][c] != 1)

    # Standard Manhattan distance
    def manhattan(self, a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    # Visualize the found path on the maze
    def visualize_path(self, path):
        display = [row[:] for row in self.maze]
        for r, c in path:
            if display[r][c] not in ('A', 'B'):
                display[r][c] = '*'
        for row in display:
            print(' '.join(str(x) for x in row))
        print()

    # Manhattan distance matrix scaled by 1.5×
    def visualize_manhattan(self):
        matrix = []
        for r in range(self.rows):
            row = []
            for c in range(self.cols):
                if self.maze[r][c] == 1:       # wall
                    row.append(None)
                else:
                    dist = self.manhattan((r, c), self.goal)
                    row.append(dist * 1.5)     # <-- scaled here
            matrix.append(row)
        return matrix

    def print_matrix(self, matrix):
        for row in matrix:
            print(' '.join(
                '.' if v is None else f"{v:4.1f}"
                for v in row
            ))
        print()


# ---------- Define the maze ----------
maze5 = [
    ['A', 0,  1,  0,  0,  0, 1, 0, 0, 1, 0, 0],
    [0,   0,  1,  0,  1,  0, 1, 0, 1, 0, 0, 0],
    [1,   0,  0,  0,  1,  0, 0, 0, 1, 1, 1, 0],
    [0,   1,  1,  0,  0,  1, 1, 0, 0, 0, 1, 0],
    [0,   0,  0,  0,  1,  0, 1, 1, 1, 0, 0, 0],
    [1,   1,  1,  0,  1,  0, 0, 0, 1, 1, 1, 0],
    [0,   0,  1,  0,  0,  0, 1, 0, 0, 0, 1, 0],
    [0,   1,  0,  1,  1,  0, 1, 1, 1, 0, 1, 0],
    [0,   0,  0,  0,  0,  0, 0, 0, 1, 0, 0, 0],
    [1,   1,  1,  1,  1,  1, 0, 1, 0, 1, 1, 0],
    [0,   0,  0,  0,  0,  0, 0, 1, 0, 0, 0, 0],
    [0,   1,  1,  1,  1,  1, 1, 1, 1, 1, 1,'B'],
]

# ---------- Run the solver ----------
solver = MazeSolver(maze5)

path = solver.greedy_bfs()
print("Path from A to B:")
print(path)
print("\nMaze with path marked (*):")
solver.visualize_path(path)

print("Manhattan Distance Matrix (scaled 1.5×):")
matrix = solver.visualize_manhattan()
solver.print_matrix(matrix)


Path from A to B:
[(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3), (6, 4), (6, 5), (5, 5), (5, 6), (5, 7), (6, 7), (6, 8), (6, 9), (7, 9), (8, 9), (8, 10), (8, 11), (9, 11), (10, 11), (11, 11)]

Maze with path marked (*):
A * 1 0 0 0 1 0 0 1 0 0
0 * 1 0 1 0 1 0 1 0 0 0
1 * * * 1 0 0 0 1 1 1 0
0 1 1 * 0 1 1 0 0 0 1 0
0 0 0 * 1 0 1 1 1 0 0 0
1 1 1 * 1 * * * 1 1 1 0
0 0 1 * * * 1 * * * 1 0
0 1 0 1 1 0 1 1 1 * 1 0
0 0 0 0 0 0 0 0 1 * * *
1 1 1 1 1 1 0 1 0 1 1 *
0 0 0 0 0 0 0 1 0 0 0 *
0 1 1 1 1 1 1 1 1 1 1 B

Manhattan Distance Matrix (scaled 1.5×):
33.0 31.5 . 28.5 27.0 25.5 . 22.5 21.0 . 18.0 16.5
31.5 30.0 . 27.0 . 24.0 . 21.0 . 18.0 16.5 15.0
. 28.5 27.0 25.5 . 22.5 21.0 19.5 . . . 13.5
28.5 . . 24.0 22.5 . . 18.0 16.5 15.0 . 12.0
27.0 25.5 24.0 22.5 . 19.5 . . . 13.5 12.0 10.5
. . . 21.0 . 18.0 16.5 15.0 . . .  9.0
24.0 22.5 . 19.5 18.0 16.5 . 13.5 12.0 10.5 .  7.5
22.5 . 19.5 . . 15.0 . . .  9.0 .  6.0
21.0 19.5 18.0 16.5 15.0 13.5 12.0 10.5 .  7.5  6.

In [4]:
import random
from heapq import heappush, heappop

class MazeSolver:
    def __init__(self, maze):
        self.maze = maze
        self.rows = len(maze)
        self.cols = len(maze[0])
        self.start = self.find_symbol('A')
        self.goal = self.find_symbol('B')

    def find_symbol(self, symbol):
        for r in range(self.rows):
            for c in range(self.cols):
                if self.maze[r][c] == symbol:
                    return (r, c)
        return None

    # ---------- Inconsistent heuristic ----------
    def inconsistent_heuristic(self, a, b):
        """
        Base Manhattan distance with a random bias that
        intentionally breaks the consistency property.
        """
        base = abs(a[0] - b[0]) + abs(a[1] - b[1])
        # Random perturbation: can be negative or > base
        bias = random.choice([-2, -1, 0, 1, 2, 3])
        return max(0, base + bias)  # keep non-negative

    # ---------- Greedy Best-First Search ----------
    def greedy_bfs(self):
        start, goal = self.start, self.goal
        frontier = []
        heappush(frontier, (self.inconsistent_heuristic(start, goal), start))
        came_from = {start: None}
        visited = set()

        while frontier:
            _, current = heappop(frontier)
            if current == goal:
                break
            if current in visited:
                continue
            visited.add(current)

            for dr, dc in [(1,0), (-1,0), (0,1), (0,-1)]:
                nr, nc = current[0] + dr, current[1] + dc
                if self.is_valid(nr, nc) and (nr, nc) not in visited:
                    h = self.inconsistent_heuristic((nr, nc), goal)
                    heappush(frontier, (h, (nr, nc)))
                    if (nr, nc) not in came_from:
                        came_from[(nr, nc)] = current

        # Reconstruct path
        path = []
        node = goal
        while node:
            path.append(node)
            node = came_from.get(node)
        return path[::-1]

    def is_valid(self, r, c):
        return (0 <= r < self.rows and 0 <= c < self.cols
                and self.maze[r][c] != 1)

    def visualize_path(self, path):
        display = [row[:] for row in self.maze]
        for r, c in path:
            if display[r][c] not in ('A', 'B'):
                display[r][c] = '*'
        for row in display:
            print(' '.join(str(x) for x in row))
        print()

    # Optional: show a matrix of the inconsistent heuristic values
    def visualize_inconsistent_matrix(self):
        matrix = []
        for r in range(self.rows):
            row = []
            for c in range(self.cols):
                if self.maze[r][c] == 1:
                    row.append(None)
                else:
                    row.append(
                        self.inconsistent_heuristic((r, c), self.goal)
                    )
            matrix.append(row)
        return matrix

    def print_matrix(self, matrix):
        for row in matrix:
            print(' '.join(
                '.' if v is None else f"{v:3}"
                for v in row
            ))
        print()


# ---------- Define the maze ----------
maze5 = [
    ['A', 0,  1,  0,  0,  0, 1, 0, 0, 1, 0, 0],
    [0,   0,  1,  0,  1,  0, 1, 0, 1, 0, 0, 0],
    [1,   0,  0,  0,  1,  0, 0, 0, 1, 1, 1, 0],
    [0,   1,  1,  0,  0,  1, 1, 0, 0, 0, 1, 0],
    [0,   0,  0,  0,  1,  0, 1, 1, 1, 0, 0, 0],
    [1,   1,  1,  0,  1,  0, 0, 0, 1, 1, 1, 0],
    [0,   0,  1,  0,  0,  0, 1, 0, 0, 0, 1, 0],
    [0,   1,  0,  1,  1,  0, 1, 1, 1, 0, 1, 0],
    [0,   0,  0,  0,  0,  0, 0, 0, 1, 0, 0, 0],
    [1,   1,  1,  1,  1,  1, 0, 1, 0, 1, 1, 0],
    [0,   0,  0,  0,  0,  0, 0, 1, 0, 0, 0, 0],
    [0,   1,  1,  1,  1,  1, 1, 1, 1, 1, 1,'B'],
]

# ---------- Run ----------
solver = MazeSolver(maze5)

print("Running Greedy BFS with INCONSISTENT heuristic...\n")
path = solver.greedy_bfs()

print("Path from A to B (may be sub-optimal due to inconsistent heuristic):")
print(path)
print("\nMaze with path marked (*):")
solver.visualize_path(path)

print("Inconsistent Heuristic Matrix (values vary each run):")
matrix = solver.visualize_inconsistent_matrix()
solver.print_matrix(matrix)


Running Greedy BFS with INCONSISTENT heuristic...

Path from A to B (may be sub-optimal due to inconsistent heuristic):
[(0, 0), (1, 0), (1, 1), (2, 1), (2, 2), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3), (6, 4), (6, 5), (5, 5), (5, 6), (5, 7), (6, 7), (6, 8), (6, 9), (7, 9), (8, 9), (8, 10), (8, 11), (9, 11), (10, 11), (11, 11)]

Maze with path marked (*):
A 0 1 0 0 0 1 0 0 1 0 0
* * 1 0 1 0 1 0 1 0 0 0
1 * * * 1 0 0 0 1 1 1 0
0 1 1 * 0 1 1 0 0 0 1 0
0 0 0 * 1 0 1 1 1 0 0 0
1 1 1 * 1 * * * 1 1 1 0
0 0 1 * * * 1 * * * 1 0
0 1 0 1 1 0 1 1 1 * 1 0
0 0 0 0 0 0 0 0 1 * * *
1 1 1 1 1 1 0 1 0 1 1 *
0 0 0 0 0 0 0 1 0 0 0 *
0 1 1 1 1 1 1 1 1 1 1 B

Inconsistent Heuristic Matrix (values vary each run):
 21  19 .  19  21  17 .  13  15 .  11   9
 22  20 .  18 .  14 .  14 .  15  12   9
.  21  18  20 .  14  15  12 . . .  12
 22 . .  15  17 . .  11  11  11 .   8
 19  17  14  18 .  15 . . .   7   6   6
. . .  13 .  15  11  11 . . .   9
 18  17 .  12  11  14 .   7   8   9 .   7
 17 .  12 . .  13 . . .   8